In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split  # Fixed 'train test split' to 'train_test_split'
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, RFE  # Fixed 'Select Best' to 'SelectKBest'
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  # Fixed 'Random ForestClassifier' to 'RandomForestClassifier'
from sklearn.decomposition import PCA
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
data = pd.read_csv(url)
data.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [32]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [33]:
if 'Age' in data.columns:
    data['Age'] = data['Age'].fillna(data['Age'].median())
if 'Embarked' in data.columns:
    data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
if 'Cabin' in data.columns:
    data = data.drop('Cabin', axis=1)


from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

data['Sex'] = label_encoder.fit_transform(data['Sex'])

data = pd.get_dummies(data, columns=['Embarked'], drop_first=True)

data.head()


PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    1  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.0      1      0   
2                             Heikkinen, Miss. Laina    0  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  35.0      1      0   
4                           Allen, Mr. William Henry    1  35.0      0      0   

             Ticket     Fare  Embarked_Q  Embarked_S  
0         A/5 21171   7.2500       False        True  
1          PC 17599  71.2833       False       False  
2  STON/O2. 3101282   7.9250       False        True  
3            113803  53.1000       False        True  
4            373450   8.0500       False        True

In [34]:

data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
data['IsAlone'] = np.where(data['FamilySize'] == 1, 1, 0)
data[['SibSp', 'Parch', 'FamilySize', 'IsAlone']].head()

SibSp  Parch  FamilySize  IsAlone
0      1      0           2        0
1      1      0           2        0
2      0      0           1        1
3      1      0           2        0
4      0      0           1        1

In [35]:
scaler = StandardScaler()
data[['Age', 'Fare', 'FamilySize']] = scaler.fit_transform(data[['Age', 'Fare', 'FamilySize']])
data[['Age', 'Fare', 'FamilySize']].head()

Age      Fare  FamilySize
0 -0.565736 -0.502445    0.059160
1  0.663861  0.786845    0.059160
2 -0.258337 -0.488854   -0.560975
3  0.433312  0.420730    0.059160
4  0.433312 -0.486337   -0.560975

In [36]:
scaler = StandardScaler()
data[['Age', 'Fare', 'FamilySize']] = scaler.fit_transform(data[['Age', 'Fare', 'FamilySize']])
data[['Age', 'Fare', 'FamilySize']].head()

Age      Fare  FamilySize
0 -0.565736 -0.502445    0.059160
1  0.663861  0.786845    0.059160
2 -0.258337 -0.488854   -0.560975
3  0.433312  0.420730    0.059160
4  0.433312 -0.486337   -0.560975

In [43]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.datasets import load_iris

data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)  
y = pd.Series(data.target)  
model = LogisticRegression(max_iter=200)

rfe = RFE(model, n_features_to_select=5)
rfe.fit(X, y)

selected_features_rfe = pd.DataFrame({
    'Feature': X.columns,
    'Selected': rfe.support_,
    'Ranking': rfe.ranking_
}).sort_values(by='Ranking')
print(selected_features_rfe)


             Feature  Selected  Ranking
0  sepal length (cm)      True        1
1   sepal width (cm)      True        1
2  petal length (cm)      True        1
3   petal width (cm)      True        1


C:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_selection\_rfe.py:291: UserWarning: Found n_features_to_select=5 > n_features=4. There will be no feature selection and all features will be kept.
  warnings.warn(


In [44]:
model = RandomForestClassifier()
model.fit(X, y)

importances = model.feature_importances_

feature_importance_rf = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance_rf)

             Feature  Importance
2  petal length (cm)    0.452160
3   petal width (cm)    0.440372
0  sepal length (cm)    0.085382
1   sepal width (cm)    0.022086


In [45]:

pca = PCA(n_components=2)

X_pca = pca.fit_transform(X)

print("Explained Variance Ratio:", pca.explained_variance_ratio_)
print('PCA Components:', X_pca[:5])



Explained Variance Ratio: [0.92461872 0.05306648]
PCA Components: [[-2.68412563  0.31939725]
 [-2.71414169 -0.17700123]
 [-2.88899057 -0.14494943]
 [-2.74534286 -0.31829898]
 [-2.72871654  0.32675451]]
